In [69]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from io import BytesIO
from IPython.display import display, HTML
import pandas as pd
from sklearn import preprocessing
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from keras.layers import Flatten

Train a LSTM to generate music 
music 21 is a module used for dealing with music data
Step1: we use a special music sheet data format which includes instruments, notes and chords called midi
We load the data from a file with a famous japanese composer joe hisaishi's music 
Using the converter.parse function of music 21, we will get a list of all notes and chords in the file.
We append the pitch of the every note object using its string notation
And we append every chord by encoding the id of chords' notes together into one string with each note seperated by a slash.
notes is the processed data. 

In [70]:
notes=[]
offsets=[]
duration=[]
composite=[]
for file in glob.glob("/Users/xiechuxi/Desktop/music/japan/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in flat 
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            composite.append([element.quarterLength])
        elif isinstance(element, chord.Chord):
            composite.append([element.quarterLength])
            
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            duration.append(element.quarterLength)
        elif isinstance(element, chord.Chord):
            duration.append(element.quarterLength)
            
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            offsets.append(element.offset)
        elif isinstance(element, chord.Chord):
            offsets.append(element.offset)
    
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('|'.join(str(n) for n in element.normalOrder))


    

Now we get a stream of notes and chords indicated by their notes as characters and chords as number string seperated by '|' (i.e. 1|6|8).
For the following step we will transfer the notes stream of charaters and chords into indices. 

In [71]:
# get all pitch names
pitchnames = sorted(set(item for item in notes))

#map the pitch names into integers
noteindices = dict((note, number) for number, note in enumerate(pitchnames))
durationvarious=sorted(set(item for item in duration))
durationdict = dict((durations,number)for number, durations in enumerate(durationvarious))
#reprocess the whole list into integer indices and seperate the list into input list and outout list we want to predict the next note after each 15 notes 
#but the sequence length could be changed to find the best tune for predict a melody
inputnetnote = []
outputnetnote = []
outputnetduration =[]
inputnetduration=[]
sequence_length = 60


for i in range(0, len(notes) - sequence_length, 1):
    sequencenotein = notes[i:i + sequence_length]
    sequencenoteout = notes[i + sequence_length]
    outputnetnote.append(noteindices[notes[i + sequence_length]])
    outputnetduration.append(durationdict[duration[i + sequence_length]])
    for char in sequencenotein:
        inputnetnote.append([noteindices[char]])
    for j in range(i,i+sequence_length):
        inputnetduration.append([durationdict[duration[j]]])
    
        
        
#reshape the input and output net into a format compatible with the LSTM net
inputnetnote = np.reshape(inputnetnote, (int(len(inputnetnote)/sequence_length), sequence_length, 1))
inputnetduration=np.reshape(inputnetduration,(int(len(inputnetduration)/sequence_length), sequence_length, 1))
#normalize the numerical value into categorical value
outputnetnote = np_utils.to_categorical(outputnetnote)

outputnetduration = np_utils.to_categorical(outputnetduration)


The function is notes,offset,duration to notes

In [72]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
import numpy as np
from keras.layers import Flatten

# split the input and output nets into test and train set


print('Build model...')
model1 = Sequential()
model1.add(LSTM(300, dropout=0.0, recurrent_dropout=0.0,input_shape=(inputnetnote.shape[1], inputnetnote.shape[2])))
model1.add(Dense(150, activation='relu'))
model1.add(Dense(len(set(notes))))
model1.add(Activation('softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=10, verbose=1, mode='auto')
print('Train...')

model1.fit(inputnetnote,outputnetnote,callbacks=[monitor],verbose=2,epochs=400)

Build model...
Train...
Epoch 1/400
 - 39s - loss: 4.3805
Epoch 2/400
 - 37s - loss: 4.0975
Epoch 3/400
 - 38s - loss: 3.9724
Epoch 4/400
 - 38s - loss: 3.8550
Epoch 5/400
 - 37s - loss: 3.7373
Epoch 6/400
 - 37s - loss: 3.6224
Epoch 7/400
 - 37s - loss: 3.5134
Epoch 8/400
 - 37s - loss: 3.4076
Epoch 9/400
 - 37s - loss: 3.3162
Epoch 10/400
 - 37s - loss: 3.2247
Epoch 11/400
 - 37s - loss: 3.1310
Epoch 12/400
 - 37s - loss: 3.0547
Epoch 13/400
 - 37s - loss: 2.9818
Epoch 14/400
 - 37s - loss: 2.8983
Epoch 15/400
 - 37s - loss: 2.8306
Epoch 16/400
 - 37s - loss: 2.7720
Epoch 17/400
 - 38s - loss: 2.7059
Epoch 18/400
 - 37s - loss: 2.6427
Epoch 19/400
 - 37s - loss: 2.5871
Epoch 20/400
 - 37s - loss: 2.5232
Epoch 21/400
 - 37s - loss: 2.4645
Epoch 22/400
 - 37s - loss: 2.4149
Epoch 23/400
 - 37s - loss: 2.3662
Epoch 24/400
 - 37s - loss: 2.3156
Epoch 25/400
 - 37s - loss: 2.2700
Epoch 26/400
 - 37s - loss: 2.2191
Epoch 27/400
 - 37s - loss: 2.1758
Epoch 28/400
 - 37s - loss: 2.1317
Epoch

The function is offset,notes,duration to duration

In [73]:
# split the input and output nets into test and train set


print('Build model...')
model2 = Sequential()
model2.add(LSTM(300, dropout=0.0, recurrent_dropout=0.0,input_shape=(inputnetduration.shape[1], inputnetduration.shape[2])))
model2.add(Dense(150, activation='relu'))
model2.add(Dense(54))
model2.add(Activation('softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=10, verbose=1, mode='auto')
print('Train...')

model2.fit(inputnetduration,outputnetduration,callbacks=[monitor],verbose=2,epochs=400)

Build model...
Train...
Epoch 1/400
 - 39s - loss: 1.7617
Epoch 2/400
 - 40s - loss: 1.5126
Epoch 3/400
 - 40s - loss: 1.2836
Epoch 4/400
 - 40s - loss: 1.1468
Epoch 5/400
 - 38s - loss: 1.0731
Epoch 6/400
 - 42s - loss: 1.0018
Epoch 7/400
 - 40s - loss: 0.9604
Epoch 8/400
 - 40s - loss: 0.9023
Epoch 9/400
 - 42s - loss: 0.8406
Epoch 10/400
 - 42s - loss: 0.7731
Epoch 11/400
 - 41s - loss: 0.7112
Epoch 12/400
 - 42s - loss: 0.6417
Epoch 13/400
 - 40s - loss: 0.5922
Epoch 14/400
 - 44s - loss: 0.5291
Epoch 15/400
 - 41s - loss: 0.4810
Epoch 16/400
 - 39s - loss: 0.4198
Epoch 17/400
 - 42s - loss: 0.3796
Epoch 18/400
 - 42s - loss: 0.3329
Epoch 19/400
 - 45s - loss: 0.2911
Epoch 20/400
 - 43s - loss: 0.2725
Epoch 21/400
 - 41s - loss: 0.2358
Epoch 22/400
 - 41s - loss: 0.2172
Epoch 23/400
 - 43s - loss: 0.2140
Epoch 24/400
 - 41s - loss: 0.1692
Epoch 25/400
 - 41s - loss: 0.1706
Epoch 26/400
 - 42s - loss: 0.1647
Epoch 27/400
 - 40s - loss: 0.1598
Epoch 28/400
 - 43s - loss: 0.1325
Epoch

The function is duration,offset to offset

In [22]:
len(durationdict)


54

In [105]:
from numpy import vstack
from fractions import Fraction
seed = np.random.randint(len(notes))
seednote=inputnetnote.flatten()
seednote=seednote[seed:seed+sequence_length]
seednote=np.reshape(seednote,(60,1))
seedduration=np.random.randint(54, size=(60,1))
#first generate a list of random notes if you want to be more creative with melody 
#you could write a small piece of notes by yourself or copy a piece online.
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
int_to_duration = dict((number, durations) for number, durations in enumerate(durationvarious))
#pattern is the iterator which each iteration will drop the first note and predict the next note according to the pattern
predictoutput=[]
#generate 500 notes as seed so that following melody could be created based on this seed
for i in range(800):
    predictinputnote = np.reshape(seednote, (1, len(seednote), 1))
    predictinputduration=np.reshape(seedduration,(1,len(seedduration),1))
    predictionnote = model1.predict(predictinputnote, verbose=0)
    predictionduration = model2.predict(predictinputduration,verbose=0)
    predictionduration1=np.argmax(predictionduration)
    index = np.argmax(predictionnote)
    result = int_to_note[index]
    resultduration=int_to_duration[predictionduration1]
    index = np.array([[index]])
    predictionduration1=np.array([[predictionduration1]])
    seednote= vstack((seednote, index))
    seednote = seednote[1:len(seednote)]
    seedduration=vstack((seedduration,predictionduration1))
    seedduration=seedduration[1:len(seedduration)]
    predictoutput.append([result,resultduration])
    

In [106]:
import math

offset = 0
output_notes = []

for l in range(0,len(predictoutput)):
    # pattern is a chord
    if ('|' in predictoutput[l][0]) or predictoutput[l][0].isdigit():
        notes_in_chord = predictoutput[l][0].split('|')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            new_note.quarterLength=predictoutput[l][1]
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(predictoutput[l][0])
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        new_note.quarterLength=predictoutput[l][1]
        #we can actually change the instrument in the further continuation of the project
        output_notes.append(new_note)
    
    offset+=0.5
    #to make sure notes don't stack

In [107]:
from music21 import *
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/Users/xiechuxi/Desktop/music/musiccreate/gn14.mid')

'/Users/xiechuxi/Desktop/music/musiccreate/gn14.mid'